# Interacción con Twitter mediante API y tweepy


La idea de esta notebook es dar algunos lineamientos sobre cómo utilizar la API de Twitter, mediante el paquete tweepy.

Antes, es necesario hacer una aclaración: la API de Twitter involucra la utilización de claves para autenticar la sesión. Para esto, es necesario a partir de una cuenta de Twitter generar una cuenta desarrolladora. La cuenta puede generarse desde [esta](https://developer.twitter.com/en) página. La idea es responder ciertas preguntas para obtener el usuario, y una vez que nos lo den, debemos crear una aplicación y ahí generar las claves.

Este proceso (sobre todo el de aceptación) puede demorar unos días. La posta es no poner palabras polémicas como bot o asociadas para que no nos lo nieguen. Poner que somos estudiantes suele ayudar.

In [ ]:
from tweepy import OAuthHandler, StreamListener, Stream, API, Cursor # Métodos para autenticación, escucha en vivo y guardado, y cuestiones generales
import time # Lo usamos para setear pausas en nuestro código
import re # Lo usamos para regular expresion

## Utilización de Clases

En general, veremos en la web que muchos de los ejemplos de utilización de la API de Twitter en Python se basa en la creación de clases de objetos que implementen funciones y objetos propios de tweepy.

En esta notebook, usamos dos sacados de [este](https://www.youtube.com/watch?v=wlnx-7cm4Gg) tutorial y uno (el primero) de elaboración propia (las diferencias saltan a la vista!).


In [ ]:
class TwitterClosetTrends():
    """
    Clase para la descarga de tendencias
    Básicamente, tendremos dos métodos asociados a esta clase:
      - get_lugaresDisponiblesParaLatLong : a partir de pasarle una ubicación, nos da información sobre qué lugares hay tendencias disponibles
      - get_tendenciasCercanas : a p API de Twitter en Python se basa enartir de pasarle una ubicación (en formato woeid : ver documentación), nos devuelve las tendencias más recientes
    Para ambos métodos, es necesario autenticarse con las claves de Twitter Developer

    """

    def __init__(self):
        self.api = API()
        auth = OAuthHandler('l5oz7nv5kXzT8xHamlmzoiLAk',
                            'fimz0i9La7sSQjmaFOmmTDLIh6yCIwMwFQTM7lFPEhCs5y5TF8') # Pasamos las api keys
        
        auth.set_access_token('1009444901703086080-4SCiPiUtrae9O2Rx9AzFGbcNSfUczn',
                              'gIxZEXZeARXUptXcCz3R1X6MqxnM3l5Ot7NsT9kCeqWpu') # Pasamos las token keys
        self.api.auth = auth
        pass
    
    def get_lugaresDisponiblesParaLatLong(self, lat, long):
        """
        Le pasamos lat, long, nos devuelve los lugares disponibles más cercanos
        """
        
        lugaresDisponibles = self.api.trends_closest(lat, long) # Usamos uno de los métodos de la api pa
        
        return lugaresDisponibles
    
    def get_tendenciasCercanas(self, woeid):
        """
        Le pasamos el woeid de alguna ubicación y nos devuelve las tendencias cercanas
        """
        tendencias_cercanas = self.api.trends_place(woeid) # Pedimos las tendencias cercanas
        
        return tendencias_cercanas


class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        pass

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list = '', languages = ['es'], follow = None):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = StdOutListener(fetched_tweets_filename)
        auth = OAuthHandler('l5oz7nv5kXzT8xHamlmzoiLAk',
                            'fimz0i9La7sSQjmaFOmmTDLIh6yCIwMwFQTM7lFPEhCs5y5TF8') # Pasamos las api keys
        
        auth.set_access_token('1009444901703086080-4SCiPiUtrae9O2Rx9AzFGbcNSfUczn',
                              'gIxZEXZeARXUptXcCz3R1X6MqxnM3l5Ot7NsT9kCeqWpu') # Pasamos las token keys
        
        stream = Stream(auth, listener, tweet_mode = 'extended')
        # This line filter Twitter Streams to capture data by the keywords: 
        if len(hash_tag_list) != 0:
            stream.filter(languages = languages,
                          track = hash_tag_list,
                          follow = follow,
                          )
        else:
            stream.sample(languages = languages,)
class StdOutListener(StreamListener):
    """
    Listener que guarda tweets en un archivo.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
    def on_error(self, status):
        print(status)

## Implementación de la Búsqueda de Tendencias

Brevemente, podemos usar la búsqueda de tendencias para saber qué es lo que más está resonando en Twitter, cercano a una determinada locación.

Esto puede permitir alimentar una búsqueda en vivo, a partir de las querys que nos otorga las propias tendencias

In [ ]:
trends = TwitterClosetTrends() # Inicializamos el objeto de tendencias
lugar = trends.get_lugaresDisponiblesParaLatLong(lat = -34, long = -58) # Encontramos la (o las) ubicación más cercana al punto indicado con lat, long de la cual Twitter tiene datos
tendencias = trends.get_tendenciasCercanas(lugar[0]['woeid']) # Pedimos las tendencias asociadas a alguno de los lugares obtenidos previamente. Según la documentación, no tiene sentido hacer esto con una frecue

print('Las tendencias serán las cercanas a {}, {}'.format(lugar[0]['name'], lugar[0]['country']))
print('Las primeras 5 tendencias son:',)
print('\n'.join([tendencias[0]['trends'][i]['name'] for i in range(20)]))

Las tendencias serán las cercanas a Buenos Aires, Argentina
Las primeras 5 tendencias son:
#EDLP
Coldplay
Texas
Coccaro
#Huracan
deportivo madryn
Acevedo
#ElVasoDeQuilmes
#eltrennosune
Nandez
#CopaArgentina
Yatra
Pozzo
De Paul
Better Call Saul
Toranzo
Cabrera
Hamburguesa o Pizza
chris martin
Dabove


In [ ]:
[tendencias[0]['trends']]

[{'name': '#EDLP',
  'promoted_content': None,
  'query': '%23EDLP',
  'tweet_volume': None,
  'url': 'http://twitter.com/search?q=%23EDLP'},
 {'name': 'Coldplay',
  'promoted_content': None,
  'query': 'Coldplay',
  'tweet_volume': 41253,
  'url': 'http://twitter.com/search?q=Coldplay'},
 {'name': 'Texas',
  'promoted_content': None,
  'query': 'Texas',
  'tweet_volume': 535095,
  'url': 'http://twitter.com/search?q=Texas'},
 {'name': 'Coccaro',
  'promoted_content': None,
  'query': 'Coccaro',
  'tweet_volume': None,
  'url': 'http://twitter.com/search?q=Coccaro'},
 {'name': '#Huracan',
  'promoted_content': None,
  'query': '%23Huracan',
  'tweet_volume': None,
  'url': 'http://twitter.com/search?q=%23Huracan'},
 {'name': 'deportivo madryn',
  'promoted_content': None,
  'query': '%22deportivo+madryn%22',
  'tweet_volume': None,
  'url': 'http://twitter.com/search?q=%22deportivo+madryn%22'},
 {'name': 'Acevedo',
  'promoted_content': None,
  'query': 'Acevedo',
  'tweet_volume': Non

### Caso de Uso Particular: Búsqueda de Tendencias para disparar Alertas


Imaginemos que queremos armar un robot que "escuche" las tendencias de Twitter, asociadas a un determinado lugar, y nos dispare algún tipo de alerta (en este caso, sólo printeará, pero podría mandarnos un mail o lo que sea) cuando alguna tendencia matchee con alguna/s palabra/s que definamos previamente

In [ ]:
def alerta_tendencias(lat, long, strings_alerta = ['vacunas'], cantidad_repeticiones = 1000, funcion = print):
    
    for i in range(cantidad_repeticiones): # Hacemos esto la cantidad de veces que definamos
        busqueda_tendencias = TwitterClosetTrends() # Inicializamos el objeto de busqueda de tendencias
        lugar = busqueda_tendencias.get_lugaresDisponiblesParaLatLong(lat = lat, long = long) # Obtenemos los lugares
        tendencias = [busqueda_tendencias.get_tendenciasCercanas(lugar[i]['woeid'])[0] for i in range(len(lugar))] # Obtenemos las tendencias. 

        for t in tendencias:
            t = t['trends']
            for trend in t:
                for string in strings_alerta:
                    if re.match('.*{}.*'.format(string.lower()), trend['name'].lower()) != None: # Criterio de matcheo de las strings de alerta y las tendencias encontradas
                        funcion(trend['name']) # Pedimos que ejecute la función que querramos en función de lo que estemos queriendo hacer
        if cantidad_repeticiones != 1:            
            time.sleep(60 * 5) # Le decimos que espere 5 minutos. Twitter actualiza la info cada 5 mins, así que no tendría sentido pedirle cada menos

# Esta es la función que pedimos que se active, pero podría ser otra tranquilamente
def print_alerta(t):
    print('Alerta! Hubo un match con: {}'.format(t))
             

In [ ]:
alerta_tendencias(lat = -34, long = -58, strings_alerta = ['Coldplay','Revolución de Mayo'],cantidad_repeticiones = 1, funcion = print_alerta)

Alerta! Hubo un match con: Coldplay
Alerta! Hubo un match con: Revolución de Mayo


## Implementación de la Búsqueda en Vivo

Veamos cómo se implementa la búsqueda de tweets en vivo:

In [ ]:
busqueda = TwitterStreamer() # Inicializamos el objeto
palabras = ['feletti'] # Definimos las palabras claves para filtrar
idioma = ['es'] # Definimos el idioma 
archivo = 'prueba.txt' # Definimos el nombre del archivo en el cual se descargan los datos

# Inicializamos la búsqueda mediante el método stream_tweets

busqueda.stream_tweets(archivo, 
                       palabras,
                       idioma,)

KeyboardInterrupt: ignored

## Trendlines de Usuarios
También podemos usar los métodos ya definidos en tweepy para buscar los últimos tweets publicados por un usuario determinado

In [ ]:
api = API()
auth = OAuthHandler('2TQBFZmx4uj1NyU7Rcg57Ouxz',
                    'Eyrcor67vCrQmZVOZ6577HAD1EQu7ilySu7HF474KWtCbrN5U6') # Pasamos las api keys

auth.set_access_token('1009444901703086080-E3EMJSfKtftv0q9aKsmhREckBnpV0T',
                      'iqZu8BHKKJs8SxASSh5TmVmKIUW8YWUEnQfiWFLdEbDPm') # Pasamos las token keys
api.auth = auth

In [ ]:
t = api.user_timeline(screen_name = 'batransito', count = 3200, exclude_replies = False, tweet_mode="extended")
print(f'Conseguimos {len(t)} tweets, desde el {t[-1]._json["created_at"]}, hasta el {t[0]._json["created_at"]}. ¿Hay más?')

Conseguimos 200 tweets, desde el Sat May 21 10:33:56 +0000 2022, hasta el Tue May 24 22:19:22 +0000 2022. ¿Hay más?


Pensémonos a nosotrxs usando Twitter: ¿todo se nos muestra? No, entramos a la app, vemos cierta cantidad de tweets, pero para ver más, tenemos que scrolear. Bueno, a la api le pasa lo mismo, tenemos que cambiar de página/ir hacia abajo. Para esto, usamos el método [Cursor](https://docs.tweepy.org/en/v4.10.0/v1_pagination.html#tweepy.Cursor)

In [ ]:
t = []

for tweet in Cursor(api.user_timeline, screen_name = 'batransito', count = 3200, exclude_replies = False, tweet_mode="extended").items():
    t.append(tweet)

print(f'Conseguimos {len(t)} tweets, desde el {t[-1]._json["created_at"]}, hasta el {t[0]._json["created_at"]}. Y ya no nos van a dar más.')

Conseguimos 3250 tweets, desde el Wed Mar 30 23:29:00 +0000 2022, hasta el Tue May 24 22:19:22 +0000 2022. Y ya no nos van a dar más.


In [ ]:
t[0]._json['user'].keys()

dict_keys(['id', 'id_str', 'name', 'screen_name', 'location', 'description', 'url', 'entities', 'protected', 'followers_count', 'friends_count', 'listed_count', 'created_at', 'favourites_count', 'utc_offset', 'time_zone', 'geo_enabled', 'verified', 'statuses_count', 'lang', 'contributors_enabled', 'is_translator', 'is_translation_enabled', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_background_tile', 'profile_image_url', 'profile_image_url_https', 'profile_banner_url', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 'has_extended_profile', 'default_profile', 'default_profile_image', 'following', 'follow_request_sent', 'notifications', 'translator_type', 'withheld_in_countries'])